In [1]:
import pandas as pd
import gensim
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import xgboost as xgb
import plotly.plotly as py
import plotly
import cufflinks as cf
import pprint
import json
import yaml

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords




paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress


In [2]:
CONFIG_FILE = '../../../../../APIKeys/API.yaml'

with open(CONFIG_FILE, 'r') as config_file:
    config = yaml.load(config_file)

PLOTLY_USERNAME = config["plotly"]["PLOTLY_USERNAME"]
PLOTLY_API_KEY = config["plotly"]["PLOTLY_API_KEY"]

plotly.tools.set_credentials_file(username = PLOTLY_USERNAME, api_key = PLOTLY_API_KEY)



In [3]:
main_data = pd.read_csv('../../business_news.csv')
main_data = main_data.sort_values(by = "Published")

news_data = main_data[["Snippet", "Lead Paragraph", "Main Headline"]]
main_data.head()

,Published,Snippet,Lead Paragraph,Main Headline,Print Headline,Seo Headline
1176,2016-10-14T04:00:00+0000,The Business Roundtable has strongly endorsed ...,The Business Roundtable has strongly endorsed ...,Corporate Board Diversity Gets Push From Busin...,NaN,NaN
1171,2017-05-15T07:21:08+0000,Regulations put in after the 2008 crisis have ...,Regulations put in after the 2008 crisis have ...,"To Spur Small Business, First Free the Banks","To Grow, First Free The Banks",NaN
1178,2017-08-28T16:33:11+0000,Emails show that Felix Sater promised to get V...,Emails show that Felix Sater promised to get V...,Trump Associate Boasted That Moscow Business D...,Trump Associate Boasted in Email Of Ties to Putin,NaN
1166,2019-01-01T10:00:06+0000,Changing your approach to traveling can go a l...,Changing your approach to traveling can go a l...,Three New Year’s Resolutions for Better Travel...,Three Resolutions For the New Year,NaN
468,2019-01-01T10:00:07+0000,"“Homecoming,” “Dirty John” and “Lore” are just...","“Homecoming,” “Dirty John” and “Lore” are just...","In the Race for Content, Hollywood Is Buying U...",From Earbud To Screen: Studios Vie For Podcasts,NaN


In [5]:
def list_prep(rows_to_list):
    news_list = rows_to_list.values.tolist()
    return(news_list)
    
list_of_lists = list_prep(news_data)
print(list_of_lists[0])



['The Business Roundtable has strongly endorsed a link between board diversity and board effectiveness and the creation of long-term shareholder value.', 'The Business Roundtable has strongly endorsed a link between board diversity and board effectiveness and the creation of long-term shareholder value.', 'Corporate Board Diversity Gets Push From Business Leaders']


In [7]:
def word_prep(list_lists):
    big_string = ""

    for title in list_lists:
    #     Splitting titles into list of words
        variable = title.split(" ")
    #     print(variable)

    #     v = words
    #     variable = list of words
        for v in variable:
            big_string = big_string + " " + v

    #     big_title_string = ' '.join((str(v) for v in variable))

    tokens = word_tokenize(big_string)

    # Remove non-alphabetic tokens, such as punctuation
    words = [word.lower() for word in tokens if word.isalpha()]

    # Filter out stopwords
    stop_words = set(stopwords.words('english'))

    words = [word for word in words if not word in stop_words]

    # Print first 10 words
    return(words)


    # print(tokens)
word_list = []
for list in list_of_lists:
    words = word_prep(list)
    word_list.append(words)
print(word_list[0])

['business', 'roundtable', 'strongly', 'endorsed', 'link', 'board', 'diversity', 'board', 'effectiveness', 'creation', 'shareholder', 'value', 'business', 'roundtable', 'strongly', 'endorsed', 'link', 'board', 'diversity', 'board', 'effectiveness', 'creation', 'shareholder', 'value', 'corporate', 'board', 'diversity', 'gets', 'push', 'business', 'leaders']


In [8]:
# Load word2vec model (trained on an enormous Google corpus)
model = gensim.models.KeyedVectors.\
load_word2vec_format('../resources/model/GoogleNews-vectors-negative300.bin', binary = True) 

# Check dimension of word vectors
model.vector_size


300

In [26]:
def word_2_vec(words):
    vector_list = []
    words_filtered = []
    # Filter the list of vectors to include only those that Word2Vec has a vector for
    vector_list.extend([model[word] for word in words if word in model.vocab])
#     return(vector_list)
    # Create a list of the words corresponding to these vectors
    words_filtered.extend([word for word in words if word in model.vocab])
#     return(words_filtered)
    # Zip the words together with their vector representations
    word_vec_zip = zip(words_filtered, vector_list)
    
    # Cast to a dict so we can turn it into a dataframe
    word_vec_dict = dict(word_vec_zip)
    return(word_vec_dict)
    


dataframe_dict_list = []
for list in word_list:
    df = word_2_vec(list)
#     print(z)
    dataframe_dict_list.append(df)


In [27]:

dataframe_list = []
for dataframe in range(len(dataframe_dict_list)):
    df = pd.DataFrame.from_dict(dataframe_dict_list[dataframe])
    dataframe_list.append(df)
# df.shape
dataframe_list[0].head()

,business,roundtable,strongly,endorsed,link,board,diversity,effectiveness,creation,shareholder,value,corporate,gets,push,leaders
0,0.010376,-0.308594,-0.058105,-0.235352,-0.077148,-0.144531,-0.214844,0.109863,-0.020874,0.070312,0.079590,-0.066895,0.075195,0.051025,0.089355
1,-0.048584,0.135742,-0.032227,-0.147461,-0.077148,-0.259766,0.166016,0.124512,-0.032715,0.193359,0.018799,0.059082,0.022827,0.168945,0.153320
2,-0.126953,-0.115234,-0.028198,0.205078,-0.036865,-0.016113,0.142578,-0.001884,-0.084473,-0.017334,0.109375,-0.231445,-0.125000,-0.131836,0.163086
3,-0.108887,0.087891,-0.055420,-0.125977,0.062256,-0.010742,0.118652,0.012817,0.012878,-0.107910,0.018677,0.101562,0.007935,-0.105469,0.233398
4,0.030273,0.251953,-0.102051,-0.218750,-0.099121,-0.012817,-0.003220,-0.024292,0.048340,0.146484,-0.114258,-0.109863,-0.050293,-0.049805,-0.142578


In [30]:
def row_summation(dataframe_list):
    summation_list = []
    for dataframe in dataframe_list:
    #     print(dataframe)
        total_columns = len(dataframe.columns)
    #     print(total_columns)
        sums = dataframe.sum(axis = 1, skipna = True)
    #     print(sums)
        summation = sums/total_columns
        summation_list.append(summation)
    return summation_list

    
summation_series = row_summation(dataframe_list)
print(summation_series[0][:5])

0   -0.042708
1    0.029647
2   -0.019681
3    0.009444
4   -0.030000
dtype: float32


In [15]:
published_list = main_data['Published'].tolist()

In [31]:
summation_dict = {}
published_index = 0
for series in summation_series:
    summation_dict.update({published_list[published_index]: series})
    published_index += 1
    
summation_df = pd.DataFrame(summation_dict)
summation_df = summation_df.T
summation_df.head()




,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
2016-10-14T04:00:00+0000,-0.042708,0.029647,-0.019681,0.009444,-0.030000,-0.058390,0.025602,-0.037046,0.146306,0.091940,...,-0.107121,-0.026034,-0.079626,0.095634,0.015284,0.013509,0.064372,0.004792,0.014158,-0.071216
2017-05-15T07:21:08+0000,0.028475,0.095679,0.023912,0.081352,-0.035714,-0.142033,-0.019288,-0.052985,0.143840,0.017806,...,-0.146977,0.012309,-0.083553,-0.017334,0.005087,0.021395,-0.095197,0.040135,0.063836,-0.102547
2017-08-28T16:33:11+0000,-0.002913,0.007083,-0.019647,0.050519,-0.048486,-0.015724,0.035422,-0.023903,0.077841,0.049966,...,0.015891,0.018854,-0.063055,-0.010825,-0.048286,-0.086072,0.034598,-0.077390,0.039180,0.016693
2019-01-01T10:00:06+0000,-0.020165,0.114699,-0.038919,0.149319,-0.029073,-0.021098,-0.019813,-0.087023,0.072137,0.042570,...,-0.100715,0.069874,-0.060326,0.026296,0.004639,0.040070,0.069238,0.006884,0.066581,-0.048204
2019-01-01T10:00:07+0000,0.062875,0.052707,-0.067801,0.108041,-0.057408,-0.058799,0.003148,-0.099071,0.143119,0.078700,...,-0.027152,0.045913,-0.095576,0.008519,0.088440,-0.048104,-0.020682,-0.091771,-0.010306,0.050415


In [22]:
summation_df.to_csv("../resources/news/business_news_word2vec.csv")


In [ ]:
from sklearn.manifold import TSNE

# Initialize t-SNE
tsne = TSNE(n_components = 2, init = 'random', random_state = 10, perplexity = 100)

# Use only 400 rows to shorten processing time
tsne_df = tsne.fit_transform(df[:400])


In [ ]:
sns.set()

fig, ax = plt.subplots(figsize = (11.7, 8.27))
sns.scatterplot(tsne_df[:, 0], tsne_df[:, 1], alpha = 0.5)

# Second plot: Import adjustText, initialize list of texts
from adjustText import adjust_text
texts = []
words_to_plot = list(np.arange(0, 400, 10))

# Append words to list
for word in words_to_plot:
    texts.append(plt.text(tsne_df[word, 0], tsne_df[word, 1], df.index[word], fontsize = 14))
    
# Plot text using adjust_text (because overlapping text is hard to read)
adjust_text(texts, force_points = 0.4, force_text = 0.4, 
            expand_points = (2,1), expand_text = (1,2),
            arrowprops = dict(arrowstyle = "-", color = 'black', lw = 0.5))

plt.show()


In [ ]:
fig = {
    'data': [
  		{
  			'x': tsne_df[:, 0], 
        	'y': tsne_df[:, 1], 
        	'text': df.index,
            "mode": "markers"
        }]}   

py.plot(fig)


In [ ]:
def document_vector(word2vec_model, doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc if word in model.vocab]
    return np.mean(model[doc], axis=0)

# Our earlier preprocessing was done when we were dealing only with word vectors
# Here, we need each document to remain a document 
def preprocess(text):
    text = text.lower()
    doc = word_tokenize(text)
    doc = [word for word in doc if word not in stop_words]
    doc = [word for word in doc if word.isalpha()] 
    return doc

# Function that will help us drop documents that have no word vectors in word2vec
def has_vector_representation(word2vec_model, doc):
    """check if at least one word of the document is in the
    word2vec dictionary"""
    return not all(word not in word2vec_model.vocab for word in doc)

# Filter out documents
def filter_docs(corpus, texts, condition_on_doc):
    """
    Filter corpus and texts given the function condition_on_doc which takes a doc. The document doc is kept if condition_on_doc(doc) is true.
    """
    number_of_docs = len(corpus)

    if texts is not None:
        texts = [text for (text, doc) in zip(texts, corpus)
                 if condition_on_doc(doc)]

    corpus = [doc for doc in corpus if condition_on_doc(doc)]

    print("{} docs removed".format(number_of_docs - len(corpus)))

    return (corpus, texts)


In [ ]:
# Preprocess the corpus
corpus = [preprocess(title) for title in titles_list]

# Remove docs that don't include any words in W2V's vocab
corpus, titles_list = filter_docs(corpus, titles_list, lambda doc: has_vector_representation(model, doc))

# Filter out any empty docs
corpus, titles_list = filter_docs(corpus, titles_list, lambda doc: (len(doc) != 0))
x = []
for doc in corpus: # append the vector for each document
    x.append(document_vector(model, doc))
    
X = np.array(x) # list to array


In [ ]:
# Initialize t-SNE
tsne = TSNE(n_components = 2, init = 'random', random_state = 10, perplexity = 100)

# Again use only 400 rows to shorten processing time
tsne_df = tsne.fit_transform(X[:400])
fig, ax = plt.subplots(figsize = (14, 10))
sns.scatterplot(tsne_df[:, 0], tsne_df[:, 1], alpha = 0.5)

texts = []
titles_to_plot = list(np.arange(0, 400, 40)) # plots every 40th title in first 400 titles

# Append words to list
for title in titles_to_plot:
    texts.append(plt.text(tsne_df[title, 0], tsne_df[title, 1], titles_list[title], fontsize = 14))
    
# Plot text using adjust_text
adjust_text(texts, force_points = 0.4, force_text = 0.4, 
            expand_points = (2,1), expand_text = (1,2),
            arrowprops = dict(arrowstyle = "-", color = 'black', lw = 0.5))

plt.show()


In [ ]:
print(titles_to_plot)
type(texts[0])

In [ ]:
fig2 = {
    'data': [
  		{
  			'x': tsne_df[:, 0], 
        	'y': tsne_df[:, 1], 
        	'text': titles_list[title],
            "mode": "markers"
        }]}   

py.plot(fig2)